In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from sklearn import datasets
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from random import randint
import seaborn as sns
import math
# Any results you write to the current directory are saved as output.

In [ ]:
# import some data to play with
iris = datasets.load_iris()

In [ ]:
iris_df = pd.DataFrame(iris['data'], columns=iris['feature_names'])
iris_df["species"] =iris.target_names[iris.target]
sns.pairplot(iris_df,hue="species")

K_max = 10
plt.show()

In [ ]:
a = []
b = []
for k in range(1,K_max+1):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(iris.data)
    a.append(kmeans.inertia_)
    if k > 1:
        b.append(silhouette_score(iris.data, kmeans.labels_, metric='euclidean'))
    
plt.plot(range(1,K_max+1),a)
plt.show()
plt.plot(range(2,K_max+1),b)
plt.show()


Implementation of a selection criterion

critere d'inertie

In [ ]:
angle_min = 5

In [ ]:
def calcInertia(X, K_max):
    '''
        returns the inertia values and the optimal k.
         The optimal K corresponds to the K after which the jump angle is almost zero (less than angle_min)
         The problem with this method comes from the fact that it is mainly graphic.
         The optimal K here generally corresponds to the elbow, that is to say to the k after which it
         there is no longer a large angle variation.
         To find the angles we calculate the cosines and to do this we reduce the dimensions to the same scale
         by dividing the ordinate data by the unit of the ordinate axis
    '''

    K_opt = 1
    a = []
    unit_ord = 1
    boolea = True
    for k in range(1,K_max+1):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(X)
        if k == 1:
            a1 = kmeans.inertia_
            #trouvons l'unite de l'axe des ordonnées
            res = a1
            while res > 1 :
                unit_ord *= 10
                res /= 10
            unit_ord /= 10
        if k== 2:
            a2 = kmeans.inertia_
            v1 = np.array([1, (a2-a1)/unit_ord])

        if k > 2:
            a1 = a2
            a2 = kmeans.inertia_
            v2 = np.array([1, (a2-a1)/unit_ord])
            cosa = np.dot(v1, v2)/(np.linalg.norm(v1) * np.linalg.norm(v2))
            angle = math.acos(cosa) * 180/math.pi
            if (abs(angle) <= angle_min) and boolea:
                K_opt = k-2
                boolea = False
            v1 = v2.copy()
        a.append(kmeans.inertia_)
    return a, K_opt

method verification

In [ ]:
_, m = calcInertia(iris.data, K_max)
print(m)

silhouette criterion

In [ ]:
def calcSilhouette(X, K_max):
    '''
        returns the silhouette values and the k which maximizes the criterion
    '''
    maxF = 0
    K_opt = 1
    a = []
    a.append(0)
    for k in range(2,K_max+1):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(X)
        val = silhouette_score(X, kmeans.labels_, metric='euclidean')
        if maxF < val:
            K_opt = k
            maxF = val
        a.append(val)
        
    return a, K_opt

In [ ]:
#alpha calculation
def calcAlphas(p):
    alphas = np.zeros(K_max - 1)
    if p==1:
        return undefined
    else:
        alphas[0] = 1 - 3/(4*p)
        for k in range(1, K_max - 1):
            alphas[k] = alphas[k-1] + (1 - alphas[k-1])/6
        return alphas
        

In [ ]:
def calculFonction(X, K_max):
    alphas = calcAlphas(X.shape[1])
    Iw = np.array([KMeans(n_clusters=k).fit(X).inertia_ for k in range(1, K_max + 1)])
    vectF = []
    minF = 1.0
    K_opt = 1
    for k in range(1, K_max + 1):
        if k==1 or Iw[k-1] == 0:
            vectF.append(1)
            val = 1
        else:
            val = Iw[k-1]/(alphas[k-2]*Iw[k-2])
            vectF.append(val)
        if minF > val:
            K_opt = k
            minF = val
    return vectF, K_opt

In [ ]:
val, i = calculFonction(iris.data, K_max)
plt.plot(range(1,K_max + 1), val)
plt.show()

1st part: simulation of the points With 3 classes and the fixed centers and evaluation of the criteria

In [ ]:
m1 = np.array([5,5])
m2 = np.array([15,10])
m3 = np.array([5,15])
v= np.array([[1,0],[0,1]])

In [ ]:
def generer():
    #valeurs générées
    data1 = np.random.multivariate_normal(size=50, mean=m1, cov=v )
    data2 = np.random.multivariate_normal(size=50, mean=m2, cov=v )
    data3 = np.random.multivariate_normal(size=50, mean=m3, cov=v )
    return data1, data2, data3

data display

In [ ]:
def plot(data):
    for i in range(len(data)):
        plt.scatter([elt[0] for elt in data[i]],[elt[1] for elt in data[i]])

In [ ]:
data1, data2, data3 = generer()
plot([data1, data2, data3])

simulation with n_iterat = 100

In [ ]:
n_iterat = 100

In [ ]:
classes_inertia = []
classes_silhouette = []
classes_fonction = []
for _ in range(n_iterat):
    data1, data2, data3 = generer()
    data = data1+data2+data3
    
    _,val = calcInertia(data, K_max)
    classes_inertia.append(val) 
    
    _,val = calcSilhouette(data, K_max)
    classes_silhouette.append(val)
    
    _,val = calculFonction(data, K_max)
    classes_fonction.append(val)
    

display of the histogram of the number of selected classes

In [ ]:
bins = [x + 0.33 for x in range(0, K_max)]
plt.hist([classes_inertia, classes_silhouette, classes_fonction], bins = bins, color = ['yellow', 'green', 'blue'],
            edgecolor = 'red', hatch = '/', label = ['Inertie', 'Silhouette_score','Critère créé'],
            histtype = 'bar') # bar est le defaut
plt.ylabel('Nombres d\'apparition')
plt.xlabel('k')
plt.title('Série')
plt.legend()

2nd part: simulation of the points With N classes and the centers generated and evaluation of the criteria
   *** Generalization of tests with N classes ***

In [ ]:
def generer(n, v, num1, num2, nb):
    '''
        generates n classes nb points (x, y) with x, y between num1 and num2, and v is the covariance matrix
    '''
    data = []
    for i in range(n):
        m = [randint(num1, num2), randint(num1, num2)]
        data.append(np.random.multivariate_normal(size=nb, mean=np.array(m), cov=v))
    return np.asarray(data)

In [ ]:
data = generer(5, v, 2, 20, 50)

In [ ]:
plot(data)

In [ ]:
def formater(data):
    #formater le ndarray
    test = []
    for elt in data:
        for i in elt:
            test.append(i.tolist())
    data2 = np.asarray(test)
    return data2

In [ ]:
classes_inertia = []
classes_silhouette = []
classes_fonction = []
for _ in range(n_iterat):
    
    data2 = formater(generer(5, v, 2, 20, 50))
    _,val = calcInertia(data2, K_max)
    classes_inertia.append(val) 
    
    _,val = calcSilhouette(data2, K_max)
    classes_silhouette.append(val)
    
    _,val = calculFonction(data2, K_max)
    classes_fonction.append(val)

display of the histogram of the number of selected classes

In [ ]:
bins = [x + 0.33 for x in range(0, K_max)]
plt.hist([classes_inertia, classes_silhouette, classes_fonction], bins = bins, color = ['yellow', 'green', 'blue'],
            edgecolor = 'red', hatch = '/', label = ['Elbow Method', 'Silhouette_score','Implemented Criterion'],
            histtype = 'bar') # bar est le defaut
plt.ylabel('Numbers of appearance')
plt.xlabel('k')
plt.title('Series')
plt.legend()